# Prévisions de la demande de vélos en libre-service
##### Projet MLDL - Roland, Lina, Maéva

### Sommaire du Projet

1. [Explication du problème de machine learning](#1-problem)
2. [Présentation du jeu de données](#2-data-presentation)
3. [Exploration du jeu de données (EDA)](#3-eda)
4. [Data cleaning et imputation](#4-cleaning)
5. [Feature Engineering](#5-feature-engineering)
6. [Préparation des données pour les modèles ML/DL](#6-preprocessing)
7. [Sélection de différents modèles](#7-model-selection)
8. [Évaluation de la performance des modèles](#8-evaluation)
9. [Discussion autour de la performance](#9-discussion)
10. [Synthèse et conclusions](#10-conclusion)

<a id="0-libs"></a>
### 0. Import des librairies nécéssaires

In [23]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns

# Configuration du style des graphiques
%matplotlib inline
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Preprocessing et Séparation
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
# Métriques d'évaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# Modèles classiques
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import warnings
import joblib  # Pour sauvegarder les modèles en .pkl
import os
warnings.filterwarnings('ignore')

<a id="1-problem"></a>
### 1. Explication du problème de machine learning

Les systèmes de vélos en libre-service, déployés aujourd'hui dans plus de 500 villes, agissent comme de véritables capteurs virtuels de la mobilité urbaine grâce à l'enregistrement précis des trajets.

C'est dans ce contexte que nous avons choisi d'analyser le **Bike Sharing Dataset**, issu du *UCI Machine Learning Repository*. Ce jeu de données regroupe l'historique des locations de vélos à Washington D.C., enrichi de **données contextuelles exogènes** essentielles telles que les conditions météorologiques (température, humidité) et les informations calendaires (jours fériés, vacances).

Dans ce projet, nous abordons une problématique de **régression supervisée sur série temporelle** visant à anticiper les flux de déplacements.

L'objectif principal est de prédire la demande horaire de vélos (variable cible `cnt`) à un horizon futur défini. Pour y parvenir, nous mettrons en œuvre et comparerons des algorithmes de **Machine Learning** (ex: XGBoost) et de **Deep Learning** (ex: LSTM). Cette démarche nous permettra de démontrer l'apport prédictif spécifique des variables contextuelles sur la performance globale des modèles.

<a id="2-data-presentation"></a>
### 2. Présentation du jeu de données

Le **Bike Sharing Dataset**, produit par le LIAAD (Université de Porto), compile l'historique complet du système *Capital Bikeshare* à Washington D.C. sur la période 2011-2012. Ce jeu de données se concentre sur le fichier `hour.csv` qui contient **17 379 observations horaires**, reliant le volume total de locations (`cnt`) à des indicateurs temporels précis (heure, jour férié, saison). Chaque enregistrement est enrichi de **données météorologiques normalisées** (température, humidité, vitesse du vent) et de conditions climatiques catégorielles (`weathersit`). 

In [24]:
df = pd.read_csv('../data/raw/hour.csv')

In [25]:
print(df.shape)
display(df.head(5))
display(df.tail(5))

(17379, 17)


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
17374,17375,2012-12-31,1,1,12,19,0,1,1,2,0.26,0.2576,0.60,0.1642,11,108,119
17375,17376,2012-12-31,1,1,12,20,0,1,1,2,0.26,0.2576,0.60,0.1642,8,81,89
17376,17377,2012-12-31,1,1,12,21,0,1,1,1,0.26,0.2576,0.60,0.1642,7,83,90
17377,17378,2012-12-31,1,1,12,22,0,1,1,1,0.26,0.2727,0.56,0.1343,13,48,61
17378,17379,2012-12-31,1,1,12,23,0,1,1,1,0.26,0.2727,0.65,0.1343,12,37,49


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


<img src="../img/2_presentation_data/dico_vars.png" width="600" align="center">

<a id="3-eda"></a>
### 3. Exploration du jeu de données

#### Visualisation sommaires

In [ ]:
# Conversion de la colonne 'dteday' en format datetime
df['dteday'] = pd.to_datetime(df['dteday'])
# Création de la variable 'time' (Date + Heure)
df['time'] = df['dteday'] + pd.to_timedelta(df['hr'], unit='h')
df = df.set_index('time')  # 'time' comme l'index du DataFrame
# Exploitation du timestamp
df['day_name'] = df['dteday'].dt.day_name()   # Nom du jour 
df['month_name'] = df['dteday'].dt.month_name() 
df['year_name'] = df['dteday'].dt.year             
print(df.index)

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00',
               '2011-01-01 02:00:00', '2011-01-01 03:00:00',
               '2011-01-01 04:00:00', '2011-01-01 05:00:00',
               '2011-01-01 06:00:00', '2011-01-01 07:00:00',
               '2011-01-01 08:00:00', '2011-01-01 09:00:00',
               ...
               '2012-12-31 14:00:00', '2012-12-31 15:00:00',
               '2012-12-31 16:00:00', '2012-12-31 17:00:00',
               '2012-12-31 18:00:00', '2012-12-31 19:00:00',
               '2012-12-31 20:00:00', '2012-12-31 21:00:00',
               '2012-12-31 22:00:00', '2012-12-31 23:00:00'],
              dtype='datetime64[ns]', name='time', length=17379, freq=None)


In [38]:
import plotly.express as px

# x=df.index permet d'utiliser ton index temporel 'time'
fig = px.line(df, x=df.index, y='cnt', title='Évolution de la demande (Cnt)')

# On affine le trait pour imiter le "linewidth=0.4"
fig.update_traces(line_width=0.5)

fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed